In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Dropout, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
tf.compat.v1.executing_eagerly_outside_functions

<function tensorflow.python.framework.ops.executing_eagerly_outside_functions()>

In [2]:
# Load data
labels = pd.read_csv("dog-breed-identification/labels.csv")
labels['file_path'] = labels['id'] + '.jpg'

In [3]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,        # Increase the rotation range
    width_shift_range=0.1,    # Decrease the width shift range
    height_shift_range=0.1,   # Decrease the height shift range
    shear_range=0.2,
    zoom_range=0.3,           # Increase the zoom range
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


In [4]:
# Split data into training and validation sets
train_data, valid_data = train_test_split(labels, test_size=0.2, random_state=42)

# Get the number of classes
num_classes = len(labels['breed'].unique())

In [5]:
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    directory="dog-breed-identification/train/",
    x_col="file_path",
    y_col="breed",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",  # 'categorical' yerine 'sparse'
    target_size=(224, 224)
)

valid_generator = train_datagen.flow_from_dataframe(
    valid_data,
    directory="dog-breed-identification/train/",
    x_col="file_path",
    y_col="breed",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",  # 'categorical' yerine 'sparse'
    target_size=(224, 224)
)

Found 6542 validated image filenames belonging to 120 classes.
Found 409 validated image filenames belonging to 120 classes.


In [6]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define your input shape
input_shape = (224, 224, 3)

# Initialize MobileNetV2 with ImageNet weights and specific input shape
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# Add your custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dense(num_classes, activation='softmax')(x)  # Assuming num_classes is the number of your output classes

# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [7]:


# Early Stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [8]:
# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=valid_generator,
    callbacks=[early_stop]
)

Epoch 1/50



InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Cihat Emre Karataş\Desktop\dog-breed\venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Cihat Emre Karataş\Desktop\dog-breed\venv\Lib\site-packages\traitlets\config\application.py", line 1077, in launch_instance

  File "C:\Users\Cihat Emre Karataş\Desktop\dog-breed\venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\Cihat Emre Karataş\Desktop\dog-breed\venv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\Cihat Emre Karataş\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\Cihat Emre Karataş\AppData\L